In [16]:
%%capture
!pip install git+https://github.com/openai/CLIP.git
!pip install deepface
!pip install face_recognition

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
!ls -la "/content/gdrive/MyDrive/"

total 431354
-rw------- 1 root root  40115107 May 24  2021  3.mp4
drwx------ 2 root root      4096 Sep 23  2021  clip_test_video
drwx------ 9 root root      4096 May 29 08:29  cluster_faces
-rw------- 1 root root 390881531 Jul 14  2021  Code.zip
drwx------ 2 root root      4096 Apr 15  2021 'Colab Notebooks'
drwx------ 2 root root      4096 Mar 13  2020  darknet
drwx------ 2 root root      4096 Oct 15  2020 'Deep Learning'
drwx------ 2 root root      4096 Apr 21  2021  demo
drwx------ 2 root root      4096 Dec 14 11:52  fk
drwx------ 2 root root      4096 Sep 23  2021  frames
-rw------- 1 root root   1560010 Mar 13  2020 'Getting started.pdf'
drwx------ 2 root root      4096 Mar 29 05:24  kissing_images
-rw------- 1 root root   9107527 May 25  2021  sub.mp4
drwx------ 2 root root      4096 Mar 10  2021  TestData


In [19]:
import torch
import clip
from PIL import Image
import time
import glob
import numpy as np
import os
from itertools import islice
import pandas as pd
import subprocess
import datetime
from deepface import DeepFace
from deepface.commons import functions
from retinaface import RetinaFace
from sklearn.cluster import DBSCAN
from imutils import build_montages
import re
from imutils import paths
import face_recognition
import pickle
import cv2
import shutil
from collections import Counter

In [20]:
class CID:

    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def load_clip(self):
        model, preprocess = clip.load("ViT-B/32", device=self.device)
        return model, preprocess

    def atoi(self, text):
        return int(text) if text.isdigit() else text

    def natural_keys(self, text):
        return [self.atoi(c) for c in re.split(r'(\d+)', text)]

    def formatTime1(self, x):
        x = str(x)
        try:
            return x.split("days")[1].strip()
        except:
            return x

    def extract_frames(self, UID, input_video):
        print(input_video)
        dest_folder_path = '/content/gdrive/MyDrive/cluster_faces/Frames/' + str(UID) + "/"
        print(dest_folder_path)
        if not os.path.exists(dest_folder_path):
            os.makedirs(dest_folder_path)
        fps = 1
        query = "ffmpeg -i " + input_video + " -pix_fmt rgb24 -vf fps=" + str(fps) + " " + dest_folder_path + "img_%06d.png"
        response = subprocess.Popen(query, shell=True, stdout=subprocess.PIPE).stdout.read()
        #subprocess.Popen(command, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        #response = subprocess.Popen(query, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        s = str(response).encode('utf-8')
        frames = []
        for file in glob.glob(dest_folder_path + '/*.png'):
            frames.append(file)
        frames.sort(key=self.natural_keys)
        print(frames)
        return frames

    def get_classes(self, celebrity_list):
        cl = []
        for i in celebrity_list:
            cl.append(f"a photo of {i}")
        cl.append("a photo of others")
        return cl

    def predict(self, fi, classes_list, model, preprocess):
        cl = classes_list
        text = clip.tokenize(cl).to(self.device)
        image = preprocess(Image.open(fi)).unsqueeze(0).to(self.device)
        with torch.no_grad():
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            probs = probs.tolist()[0]
        vv = {}
        for i, j in enumerate(probs):
            vv[cl[i]] = j
        maxx = {k: v for k, v in sorted(vv.items(), key=lambda item: item[1], reverse=True)}
        tmp = list(islice(maxx.items(), 3))
        nn = fi.split('/')
        nn = nn[len(nn) - 1]
        print(f"{nn} => {tmp}")
        print('_____________________')
        return tmp

    def get_similarity(self, img1, img2):
        result = DeepFace.verify(img1_path=img1, img2_path=img2, detector_backend='skip', enforce_detection=False)
        return result['distance']

    def seconds2timestamp(self, sec):
        timestamp = datetime.timedelta(seconds=sec)
        return timestamp

    def decodePrediction(self, c1, s1):
        if c1 != 'a photo of others' or c1 != "a photo of an object":
            if s1 > 0.4:
                return "person"
        return "no_person"

    def timeStamp2seconds(self, timestring):
        pt = datetime.datetime.strptime(timestring, '%H:%M:%S')
        total_seconds = pt.second + pt.minute * 60 + pt.hour * 3600
        return total_seconds

    def checkStringPresent(self, wstring, sub_str):
        if wstring.find(sub_str) == -1:
            return False
        else:
            return True

    def get_images_with_person(self, frames_list, model, preprocess):
        persons_list = []
        classes_list = ["a photo of a person", "a photo of group of people", "a photo of other",
                        "a photo of a masked person", "a photo of two or more people", "a photo of a man",
                        "a photo of a woman", "a photo of a man and a women",
                        "a photo of an object"]
        for ind, fi in enumerate(frames_list):
            try:
                tmp = []
                Highest3Predictions = self.predict(fi, classes_list, model, preprocess)
                c1 = Highest3Predictions[0][0]
                s1 = Highest3Predictions[0][1]
                status = self.decodePrediction(c1, s1)
                if status == "person":
                    persons_list.append(fi)
            except:
                pass

        return persons_list

    def get_embeddings(self, img_paths, model):
        instances = {}
        for img_path in img_paths:
            try:
                img = functions.preprocess_face(img_path, target_size=(160, 160))
                # represent
                embeding = model.predict(img)[0].tolist()
                # raw image name without path and extension
                # label = img_path.split("/")[-1].split(".")[0]
                instances[img_path] = embeding
            except:
                pass
        return instances

    def extractFaces(self, images_path, id):
        saveFacesHere = '/content/gdrive/MyDrive/cluster_faces/Faces/' + str(id) + '/'
        saveFacesHere_forCLIP = '/content/gdrive/MyDrive/cluster_faces/Faces_Clip/' + str(id) + '/'
        if not os.path.exists(saveFacesHere):
            os.makedirs(saveFacesHere)
        if not os.path.exists(saveFacesHere_forCLIP):
            os.makedirs(saveFacesHere_forCLIP)
        df = pd.DataFrame(columns=["FacesPath", "FacesPathCLIP", "ImagePath", "TimeStamp", "FA1", "FA2", "FA3", "FA4"])

        # z = 0
        for image_path in images_path:
            z = image_path.split("/")
            z = z[len(z) - 1].split(".")[0]
            try:
                resp = RetinaFace.detect_faces(image_path)
                img = cv2.imread(image_path)
                image_wid = img.shape[1]
                image_hgt = img.shape[0]
                i = 0
                for key, value in resp.items():
                    tmp = []
                    aa = value['facial_area']
                    FA = [aa[1], aa[2], aa[3], aa[0]]
                    distnce_between_rightleft_eye = abs(
                        value['landmarks']['right_eye'][0] - value['landmarks']['left_eye'][0])
                    if distnce_between_rightleft_eye < 25:
                        pass
                    else:
                        x1, y1, x2, y2 = aa[0], aa[1], aa[2], aa[3]
                        x = x1
                        y = y1
                        w = abs(x2 - x1)
                        h = abs(y2 - y1)
                        crop_img = img[y:y + h, x:x + w]
                        wid = crop_img.shape[1]
                        hgt = crop_img.shape[0]

                        if (x + w + 50) <= image_wid:
                            croped_hight = y + h + 50
                        else:
                            croped_hight = y + h
                        if (y + h + 50) <= image_hgt:
                            croped_width = x + w + 50
                        else:
                            croped_width = x + w
                        crop_img_clip = img[y - 30:croped_hight, x - 30:croped_width]
                        if abs(wid - hgt) < 15:
                            pass
                        else:
                            cv2.imwrite(saveFacesHere + str(z) + "_" + str(i) + '.png', crop_img)
                            try:
                                cv2.imwrite(saveFacesHere_forCLIP + str(z) + "_" + str(i) + '.png', crop_img_clip)
                            except:
                                cv2.imwrite(saveFacesHere_forCLIP + str(z) + "_" + str(i) + '.png', crop_img)
                            tmp.append(saveFacesHere + str(z) + "_" + str(i) + '.png')
                            tmp.append(saveFacesHere_forCLIP + str(z) + "_" + str(i) + '.png')
                            tmp.append(image_path)
                            tmp.append(self.dfsec2t(z))
                            tmp.append(FA[0])
                            tmp.append(FA[1])
                            tmp.append(FA[2])
                            tmp.append(FA[3])
                            i += 1
                    if len(tmp) != 0:
                        df_length1 = len(df)
                        df.loc[df_length1] = tmp
            except Exception as e:
                # res[image_path] = []
                print('Error in cropping face :', e)

        return df

    def get_cluster(self, encodings_file_path, UID, data1):
        dest_folder_path = "/content/gdrive/MyDrive/cluster_faces/ClusterImage/" + str(UID) + "/"
        if not os.path.exists(dest_folder_path):
            os.makedirs(dest_folder_path)
        df = pd.DataFrame(columns=['Character ID', 'ImagePath'])
        print("[INFO] loading encodings...")
        data = pickle.loads(open(encodings_file_path, "rb").read())
        data = np.array(data)
        print(len(data))

        if len(data) <= 200:
            ms = 3
            ep = 0.5
        elif 200 < len(data) <= 300:
            ms = 5
            ep = 0.45
        elif 300 < len(data) <= 500:
            ms = 8
            ep = 0.42
        else:
            ms = 10
            ep = 0.39

        encodings = [d["encoding"] for d in data]
        nn = [ss["imagePath"] for ss in data1]
        # print(nn)
        # cluster the embeddings
        print("[INFO] clustering...")
        # clt = DBSCAN(metric="euclidean", n_jobs=args["jobs"])
        # eps=3, min_samples=2
        # algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
        # leaf_sizeint, default=30
        # clt = DBSCAN(metric="euclidean", n_jobs=-1, eps=0.39, min_samples=24)
        clt = DBSCAN(metric="euclidean", n_jobs=-1, eps=ep, min_samples=ms)
        clt.fit(encodings)

        # determine the total number of unique faces found in the dataset
        labelIDs = np.unique(clt.labels_)
        numUniqueFaces = len(np.where(labelIDs > -1)[0])
        print("[INFO] # unique faces: {}".format(numUniqueFaces))
        # print(labelIDs)
        # print(numUniqueFaces)

        # loop over the unique face integers
        for labelID in labelIDs:
            # find all indexes into the `data` array that belong to the
            # current label ID, then randomly sample a maximum of 25 indexes
            # from the set
            print("[INFO] faces for face ID: {}".format(labelID))
            idxs = np.where(clt.labels_ == labelID)[0]
            a = []
            for i in idxs:
                a.append(nn[i])
            # print(idxs)
            a.sort(key=self.natural_keys)
            # print(a)
            tit = "Face ID #{}".format(labelID)
            ti = "Not Clustered Faces" if labelID == -1 else tit
            for zz in a:
                tmp = [ti, zz]
                df_length1 = len(df)
                df.loc[df_length1] = tmp

            idxs = np.random.choice(idxs, size=min(100, len(idxs)),
                                    replace=False)

            # initialize the list of faces to include in the montage
            faces = []

            # loop over the sampled indexes
            for i in idxs:
                # load the input image and extract the face ROI
                try:
                    image = cv2.imread(data[i]["imagePath"])
                    (top, right, bottom, left) = data[i]["loc"]
                    face = image[top:bottom, left:right]

                    # force resize the face ROI to 96x96 and then add it to the
                    # faces montage list
                    face = cv2.resize(face, (112, 112))
                    faces.append(face)
                except:
                    pass

            # create a montage using 96x96 "tiles" with 5 rows and 5 columns
            # print(faces)
            montage = build_montages(faces, (112, 112), (10, 15))[0]

            # show the output montage
            title = "Face ID #{}".format(labelID)
            title = "Unknown Faces" if labelID == -1 else title
            cv2.imwrite(dest_folder_path + str(title) + ".png", montage)
            # cv2.imshow(title, montage)
            # cv2.waitKey(0)

        return df

    def encode_faces(self, df, UID):
        # for (i, imagePath) in enumerate(imagePaths):
        data = []
        data1 = []
        pickle_file_path = "/content/gdrive/MyDrive/cluster_faces/pickle_files/" + str(UID) + ".pickle"
        for ind, row in df.iterrows():
            try:
                # load the input image and convert it from RGB (OpenCV ordering)
                # to dlib ordering (RGB)
                print("[INFO] processing image {}/{}".format(ind + 1, len(df)))
                # print(imagePath)
                image = cv2.imread(row['ImagePath'])
                rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # detect the (x, y)-coordinates of the bounding boxes
                # corresponding to each face in the input image
                # boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
                boxes = [(int(row['FA1']), int(row['FA2']), int(row['FA3']), int(row['FA4']))]

                # compute the facial embedding for the face
                encodings = face_recognition.face_encodings(rgb, boxes)
                # Len of encodings : 128 d-vector

                # build a dictionary of the image path, bounding box location,
                # and facial encodings for the current image
                d = [{"imagePath": row['FacesPathCLIP'], "loc": box, "encoding": enc}
                     for (box, enc) in zip(boxes, encodings)]
                data.extend(d)

                d1 = [{"imagePath": row['ImagePath'], "loc": box, "encoding": enc}
                      for (box, enc) in zip(boxes, encodings)]
                data1.extend(d1)

            except Exception as e:
                print(row)
                print("Exception in Encoding", e)
                pass
        print("[INFO] serializing encodings...")
        f = open(pickle_file_path, "wb")
        f.write(pickle.dumps(data1))
        f.close()

        f1 = open("/content/gdrive/MyDrive/cluster_faces/pickle_files/" + str(UID) + "_1.pickle", "wb")
        f1.write(pickle.dumps(data))
        f1.close()
        return pickle_file_path, data

    def decode_actor_actress(self, c1, s1):
        if s1 >= 0.60:
            na = c1.split("a photo of ")[1]
            return na, s1, True
        else:
            na = c1.split("a photo of ")[1]
            return na, s1, False

    def format_line1(self, FrameName, p1, p2):
        dd = str(FrameName) + '\t' + str(p1) + '\t' + str(p2) + '\n'
        return dd

    def format_line(self, UID, type, time):
        dd = str(UID) + '\t' + str(type) + '\t' + str(time) + '\n'
        return dd

    def get_celebrity(self, frame, actor_actress_list, model, preprocess):
        Highest3Predictions = self.predict(frame, actor_actress_list, model, preprocess)
        c1 = Highest3Predictions[0][0]
        s1 = Highest3Predictions[0][1]
        c, s, st = self.decode_actor_actress(c1, s1)

        if st:
            return c
        else:
            return "unknown"

    def ti2sec(self, x):
        try:
            prev = datetime.datetime.strptime(str(x), '%H:%M:%S')
            prev = prev.second + prev.minute * 60 + prev.hour * 3600
            return prev
        except:
            return x

    def dfsec2t(self, a):
        try:
            a = int(a.split('_')[1])
            return self.seconds2timestamp(a)
        except:
            pass

    def formatTime(self, x):
        x = str(x)
        try:
            return x.split("days")[1].strip()
        except:
            return x

    def cluster_using_clip(self, UID, actor_actress_list, clip_model, preprocess):
        extract_faces_df = pd.read_csv("/content/gdrive/MyDrive/cluster_faces/DF/" + str(UID) + "_FacesExtract.csv")
        extract_faces_df["celebrity"] = " "
        for ind, row in extract_faces_df.iterrows():
            try:
                celeb = self.get_celebrity(row["FacesPathCLIP"], actor_actress_list, clip_model, preprocess)
                extract_faces_df["celebrity"].iloc[ind] = celeb
            except:
                extract_faces_df["celebrity"].iloc[ind] = "unknown"

        grouped_df = extract_faces_df.groupby(['celebrity'])
        s = []
        for key, item in grouped_df:
            a = grouped_df.get_group(key)
            s.append(a)
            # print(grouped_df.get_group(key), "\n\n")
        sss = pd.concat(s, ignore_index=True)
        sss["Character ID"] = ""
        ll = sss["celebrity"].tolist()
        prev = ll[0]
        a = 0
        for i, j in enumerate(ll):
            if i == 0:
                prev = ll[i]
                sss["Character ID"].iloc[i] = "Face ID #" + str(a)
            else:
                curr = ll[i]
                b = a
                if prev == curr:
                    if prev == "unknown":
                        sss["Character ID"].iloc[i] = "Not Clustered Faces"
                    else:
                        sss["Character ID"].iloc[i] = "Face ID #" + str(a)
                    # prev = curr
                else:
                    b += 1
                    if curr == "unknown":
                        sss["Character ID"].iloc[i] = "Not Clustered Faces"
                    else:
                        sss["Character ID"].iloc[i] = "Face ID #" + str(b)
                    a = b
                    prev = curr
        col = ["Character ID", "celebrity", "FacesPath", "ImagePath", "TimeStamp"]
        sss = sss[col]
        sss.columns = ['Character ID', 'Celebrity', 'FacePath', 'ImagePath', "TimeStamp"]
        return sss
        # extract_faces_df.to_csv("xxxx.csv")
        # print(extract_faces_df)

    def formatDF(self, df):
        try:
            df = df[(df['GTClassification'] != "neutral")]
            df["TimeStamp"] = df["Seconds"].apply(self.seconds2timestamp).apply(self.formatTime)
            return df
        except:
            return df

    def start_process(self, UID, celebrity_list, input_video_path):
        frames_list = self.extract_frames(UID, input_video_path)
        # frames_list = [FramesPath + 'img_' + str(self.ti2sec(i)).zfill(6) + '.png' for i in tim]
        # frames_list = list(set(frames_list))
        frames_list.sort(key=self.natural_keys)
        #
        clip_model, preprocess = self.load_clip()
        frames_with_person = self.get_images_with_person(frames_list, clip_model, preprocess)
        #
        extract_faces_df = self.extractFaces(frames_with_person, UID)
        extract_faces_df["TimeStamp"] = extract_faces_df["TimeStamp"].apply(self.formatTime)
        extract_faces_df.to_csv('/content/gdrive/My Drive/cluster_faces/DF/' + str(UID) + "_FacesExtract" + '.csv')
        print(extract_faces_df)
        #
        pickle_file_path, data = self.encode_faces(extract_faces_df, UID)
        print(pickle_file_path)
        #
        df = self.get_cluster(pickle_file_path, UID, data)
        df["FacePath"] = " "
        df["TimeStamp"] = " "
        df["Celebrity"] = " "
        if len(celebrity_list) != 0:
            actor_actress_list = self.get_classes(celebrity_list)
        else:
            actor_actress_list = []
        for ind, row in df.iterrows():
            try:
                aa = extract_faces_df[(extract_faces_df['FacesPathCLIP'] == row["ImagePath"])]["ImagePath"].tolist()[0]
                face_path = extract_faces_df['FacesPathCLIP'].iloc[ind]
                # bb = extract_faces_df['TimeStamp'].iloc[ind]
                df["FacePath"].iloc[ind] = aa
                bc = extract_faces_df[(extract_faces_df['ImagePath'] == row["FacePath"])]["TimeStamp"].tolist()[0]
                df["TimeStamp"].iloc[ind] = bc
                # df["TimeStamp"].iloc[ind] = bb
                if len(actor_actress_list) != 0:
                    # celeb = self.get_celebrity(face_path, actor_actress_list, clip_model, preprocess)
                    df["Celebrity"].iloc[ind] = "unknown"
                else:
                    df["Celebrity"].iloc[ind] = "unknown"
            except:
                pass
        #
        col = ["Character ID", "Celebrity", "FacePath", "ImagePath", "TimeStamp"]
        df = df[col]
        df.columns = ['Character ID', 'Celebrity', 'ImagePath', 'FacePath', "TimeStamp"]

        df.to_csv('/content/gdrive/My Drive/cluster_faces/DF/' + str(UID) + "_cluster" + '.csv')
        print(df)

        if len(celebrity_list) != 0:
            actor_actress_list = self.get_classes(celebrity_list)
            sss = self.cluster_using_clip(UID, actor_actress_list, clip_model, preprocess)
            sss.to_csv('/content/gdrive/My Drive/cluster_faces/CLIP_DF/' + str(UID) + "_cluster" + '.csv')
        else:
            actor_actress_list = []
            # pickle_file_path, data = self.encode_faces(extract_faces_df, UID)
            # print(pickle_file_path)
            # df = self.get_cluster(pickle_file_path, UID, data)

  

In [15]:
obj = CID()

celebrity_list = ["Leonardo DiCaprio","Mark Ruffalo","Ben Kingsley","Michelle Williams","Emily Mortimer","Patricia Clarkson","Max von Sydow"]
obj.start_process(77, celebrity_list, "/content/gdrive/MyDrive/cluster_faces/Shutter_Island.mp4")

/content/gdrive/MyDrive/cluster_faces/Shutter_Island.mp4
/content/gdrive/MyDrive/cluster_faces/Frames/77/
['/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000001.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000002.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000003.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000004.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000005.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000006.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000007.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000008.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000009.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000010.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000011.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000012.png', '/content/gdrive/MyDrive/cluster_faces/Frames/77/img_000013.png', '/content/gdrive/MyDrive/cluster_fa

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 59.1MiB/s]


img_000001.png => [('a photo of a man', 0.414306640625), ('a photo of other', 0.208251953125), ('a photo of a man and a women', 0.1500244140625)]
_____________________
img_000002.png => [('a photo of a man', 0.360107421875), ('a photo of a man and a women', 0.201904296875), ('a photo of other', 0.17822265625)]
_____________________
img_000003.png => [('a photo of a man', 0.277099609375), ('a photo of a man and a women', 0.2261962890625), ('a photo of other', 0.21240234375)]
_____________________
img_000004.png => [('a photo of other', 0.312255859375), ('a photo of a man', 0.262939453125), ('a photo of a man and a women', 0.1697998046875)]
_____________________
img_000005.png => [('a photo of two or more people', 0.22412109375), ('a photo of group of people', 0.1947021484375), ('a photo of other', 0.188720703125)]
_____________________
img_000006.png => [('a photo of two or more people', 0.276123046875), ('a photo of other', 0.2254638671875), ('a photo of a man and a women', 0.183959960

Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:31<00:00, 3.75MB/s]


Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
Error in cropping face : 'tuple' object has no attribute 'items'
                                            FacesPath  \
0   /content/gdrive/MyDrive/cluster_faces/Faces/77...   
1   /content/gdrive/MyDrive/cluster_faces/Faces/77...   
2   /content/gdrive/MyDrive/cluster_faces/Faces/77...   
3   /content/gdrive/MyDrive/cluster_faces/Faces/77...   
4   /content/gdrive/MyDrive/cluster_faces/Faces/77...   
..                                                ...   
70  /content/gdr

In [ ]:
obj = CID()
celebrity_list = ["Robert Downey Jr.", "Chris Evans", "Mark Ruffalo", "Chris Hemsworth", "Scarlett Johansson","Jeremy Renner",
                  "Tom Hiddleston","Clark Gregg","Cobie Smulders","Stellan Skarsgård",
                    "Samuel L. Jackson"]

obj.start_process(78, celebrity_list, "/content/gdrive/MyDrive/cluster_faces/avengers.mp4")
